In [0]:
import requests
import pandas as pd
from pyspark.sql.types import StructType, StructField, StringType

headers = {
    'accept': 'application/json',
    'x-access-token-ws': 'd1bf4238656c2167c5997e4081b549eedd0a25622a18740d8ad1a8a31ee22b15'
}

# -------------------- EXTRAÇÃO ---------------------------------------
url = 'https://api.irancho.com.br/api/fazenda'
response = requests.get(url, headers=headers)
fazenda_list = response.json()
df_fazenda_pandas = pd.json_normalize(fazenda_list, sep='.')
print(f'fazenda {response.status_code}')

# ✅ CONVERTER TUDO PARA STRING NO PANDAS
df_fazenda_pandas = df_fazenda_pandas.astype(str)

# Criar schema com tudo como String
schema = StructType([
    StructField(col, StringType(), True) 
    for col in df_fazenda_pandas.columns
])

df_fazenda = spark.createDataFrame(df_fazenda_pandas, schema=schema)
df_fazenda.write.mode("overwrite").saveAsTable("fazenda_raw")

print("✅ Tabela criada com sucesso!")

In [0]:
%sql
SELECT * FROM fazenda_raw

In [0]:
%sql
DROP SCHEMA WORKSPACE.IRANCHO;

In [0]:
%sql
CREATE OR REPLACE table WORKSPACE.IRANCHO.FAZENDAS AS 
  SELECT NOME, 
      ID_FAZENDA, 
      ID_ASSINATURA,
      ID_PAIS,
      ID_ESTADO, 
      ID_MUNICIPIO, 
      ID_PRODUTOR, 
      SG_FAZENDA, 
      ED_FAZENDA, 
      NU_INCRA, 
      NU_RECEITA_FEDERAL, 
      IC_SISBOV_FAZENDA, 
      NU_SISBOV_FAZENDA, 
      NU_CPF_CNPJ, 
      QT_AREA_TOTAL, 
      QT_AREA_RESERVA, 
      ATIVO, 
      ID_ENTIDADE_CERTIFICADORA, 
      LATITUDE, 
      LONGITUDE, 
      RAZAO_SOCIAL, 
      DISTANCIA_MUNICIPIO, 
      VALOR_TERRA_HA, 
      `unidade_medida.no_unidade_medida` as UNIDADE_MEDIDA, /*UTILIZE CRAZE PARA COLUNAS ANINHADAS*/
      `estado.nome` as ESTADO, 
      `municipio.nome` as MUNICIPIO, 
      dt_alteracao
  FROM fazenda_raw;

In [0]:
%sql
SELECT * FROM irancho.fazendas